In [1]:
from google.colab import drive
import os, sys
from datetime import datetime
import numpy as np
from tqdm.notebook import tqdm 
import tensorflow as tf
from pytz import timezone
from glob import glob 
import matplotlib.pyplot as plt
import shutil
import cv2


drive.mount('/content/drive')
link_name = '/packages'
target = '/content/drive/My Drive/Colab Notebooks/collage/packages'

try:
  os.symlink(target, link_name)
except:
  os.chdir(target)
  os.remove(link_name)
  os.symlink(target, link_name)
sys.path.insert(0, link_name)

from pororo import Pororo
os.chdir('/content/drive/MyDrive/Colab Notebooks/collage')
import scrap
import cairosvg 
import segsave
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36
# Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
input user-agent 
Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36


In [2]:
# # !pip --version
# # !pip install --target=$link_name --upgrade pip
# # !pip install --target=$link_name --use-deprecated=legacy-resolver git+https://github.com/kakaobrain/pororo.git
# # !pip install --target=$link_name konlpy
# # !pip install --target=$link_name python-mecab-ko
# # !pip install --target=$link_name kollocate
# # !pip install --target=$link_name koparadigm
# # !pip install --target=$link_name g2pk
# # !pip install --target=$link_name selenium
# # !pip install --target=$link_name webdriver_manager
# # !pip install --target=$link_name git+https://github.com/naver/kor2vec.git
# # !pip install  --target=$link_name cairosvg
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency 

In [3]:
def return_trend_dict():
  return {'keyword': None, 'namuKeyword': None, 'namuKeyword_kind': None, 'articles': None, 'article_summary':None, 'img_name':None, 'img':None, 'links':None, 'title_picked':None}

In [4]:
now = datetime.now(timezone('Asia/Seoul'))
datetime_text = str(now.strftime('%y-%m-%d-%H:%M'))

keyword_dicts = []
keyword_now = scrap.scrapSignalKeyword()
# keyword_now = ['이상한 변호사 우영우', '하락 서울 6개월만에', '문지효', '담대한 어리석음 상대', '수사한 이원석 총장에', '이상민', '박시은', '트로트', '송파구청장', 'MBC 뉴스데스크']
# keyword_now = ['이상한 변호사 우영우', '강제 북송 검찰', '박시은', '정경심 허리 디스크', '메이플', '정창욱 징역 1년', '이상민', '나는솔로', '이준석', '히든싱어 7 송은이']

print(f'{datetime_text} : {keyword_now}')
for idx, keyword in tqdm(enumerate(keyword_now)):
  dict_ = return_trend_dict()
  articles, links = scrap.scrapNaverNewsKeyword(keyword, article_num=10, sort='1')
  dict_['keyword'] = keyword
  dict_['articles'] = articles
  dict_['img_name'] = str(idx) + '.png'
  dict_['links'] = links
  keyword_dicts.append(dict_)

22-08-20-11:34 : ['코로나19 12만', '진태현', '박시은', '블랙핑크 69개국 1위', '이재명', '황희찬', '전 여친 협박한', '일라이', '이준석', '한지혜']


0it [00:00, ?it/s]

In [5]:
def person_replace(txt):
  per_dict = { '尹':'윤석열', '文':'문'}
  for word, replacement in per_dict.items():
    txt = txt.replace(word, replacement)
  return txt

In [6]:
ner = Pororo(task="ner", lang="ko")
se = Pororo(task="sentence_embedding")
# we = Pororo(task="word_embedding")

In [7]:
def find_namukey(keyword, d=False):
  ner_result = ner(person_replace(keyword))
  ner_person_list = [re[0] for re in ner_result if re[1]=='PERSON']
  ner_or_list = [re[0] for re in ner_result if re[1]=='ORGANIZATION']
  ner_arti_list = [re[0] for re in ner_result if re[1]=='ARTIFACT']
  ner_city_list = [re[0] for re in ner_result if re[1]=='CITY']

  if d:
    if len(ner_person_list) > 0:
      for p in ner_person_list:
        if keyword == p:
          return p, 'person'
      return ner_person_list[0], 'person'

    elif len(ner_or_list) > 0:
      for o in ner_or_list:
        if keyword == o:
          return o, 'organization'
      return ner_or_list[0], 'organization'

    elif len(ner_arti_list) > 0:
      for a in ner_arti_list:
        if keyword == a:
          return a, 'artifact'
      return ner_arti_list[0], 'artifact'

    elif len(ner_city_list) > 0:
      for c in ner_city_list:
        if keyword == c:
          return c, 'city'
      return ner_city_list[0], 'city'
    else :
      return '-', '-'
  else:
    if len(ner_person_list) > 0:
      for p in ner_person_list:
        if keyword == p:
          return p, 'person'
      return ner_person_list[0], 'person'
    else :
      return '-', '-'

In [8]:
for dict_ in tqdm(keyword_dicts):
  dict_['namuKeyword'], dict_['namuKeyword_kind'] = find_namukey(dict_['keyword'])

  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
print([k['namuKeyword'] for k in keyword_dicts])

['-', '진태현', '박시은', '블랙핑크', '이재명', '황희찬', '-', '일라이', '이준석', '한지혜']


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def find_impor_title_idx(dict_):
  titles = [article[0].strip() for article in dict_['articles']]
  titles_embedding = [se(t).reshape(1,-1) for t in titles]

  sims = []
  for i in range(len(titles_embedding)):
    cands = titles_embedding.copy()
    query = cands.pop(i)
    temp_num = 0
    for c in cands:
      temp_num += cosine_similarity(query, c)[0][0]
    sims.append(temp_num)
  max_ = max(sims)
  max_idx = sims.index(max_)

  return max_idx

In [11]:
for dict_ in tqdm(keyword_dicts):
  max_idx = find_impor_title_idx(dict_)
  dict_['title_picked'] = max_idx
  if dict_['namuKeyword']=='-':
    target_title = dict_['articles'][max_idx][0]
    dict_['namuKeyword'], dict_['namuKeyword_kind'] = find_namukey(target_title, d=True)

  0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
print([k['namuKeyword'] for k in keyword_dicts])

['-', '진태현', '박시은', '블랙핑크', '이재명', '황희찬', '가세연', '일라이', '이준석', '한지혜']


In [13]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/collage/result')

for i,dict_ in enumerate(keyword_dicts):
    print(f"{dict_['keyword']} -> {dict_['namuKeyword']}")
    try:
        os.mkdir('./'+datetime_text)
    except:
        pass
    try:
        os.mkdir(f'./{datetime_text}/img')
        os.mkdir(f'./{datetime_text}/txt')
    except:
        pass
    try:
        os.mkdir(f"./{datetime_text}/txt/{dict_['keyword']}")
    except:
        pass
    txt_path = f"./{datetime_text}/txt/{dict_['keyword']}"
    # scrap.saveArticles(dict_['articles'], txt_path)
    img_path = f'./{datetime_text}/img/{i}.png'
    scrap.scrapNamuImg(dict_['namuKeyword'],img_path, namuKeyword_kind=dict_['namuKeyword_kind'])

코로나19 12만 -> -
진태현 -> 진태현
박시은 -> 박시은
블랙핑크 69개국 1위 -> 블랙핑크
이재명 -> 이재명
황희찬 -> 황희찬
전 여친 협박한 -> 가세연
일라이 -> 일라이
이준석 -> 이준석
한지혜 -> 한지혜


In [14]:
im_list = glob(f'./{datetime_text}/img/*')
try:
    os.mkdir('./'+datetime_text+'/segmentation')
except:
    print('-')
model_present_path = '/content/drive/MyDrive/Colab Notebooks/collage/segmentation/model_weight/base_weight.h5'
model_present = tf.keras.models.load_model(model_present_path)

In [15]:
for dict_ in tqdm(keyword_dicts):
  load_path = f'./{datetime_text}/img/{dict_["img_name"]}'
  save_path = f'./{datetime_text}/segmentation/{load_path.split("/")[-1]}'
  if dict_['namuKeyword_kind'] == 'person':
    try:
      temp_img = segsave.segSave(model_present, load_path, save_path=None)
      temp_img = np.stack([temp_img[:,:,2], temp_img[:,:,1], temp_img[:,:,0], temp_img[:,:,3]], axis=2)
      dict_['img'] = segsave.makeWhiteEdge(temp_img, save_path=save_path, iterations=5)
    except:
      pass
  elif dict_['namuKeyword_kind'] == '-':
    continue   
  else:
    try:
      temp_img = cv2.imread(load_path, cv2.IMREAD_UNCHANGED)
      temp_img = np.stack([temp_img[:,:,2], temp_img[:,:,1], temp_img[:,:,0], temp_img[:,:,3]], axis=2)
      try:
        dict_['img'] = segsave.makeWhiteEdge(temp_img, save_path=save_path, iterations=5)
      except:
        shutil.copy(load_path, save_path)
        dict_['img'] = temp_img
    except:
      pass

  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
keyword_dicts[0]

{'keyword': '코로나19 12만',
 'namuKeyword': '-',
 'namuKeyword_kind': '-',
 'articles': [('[속보] 위중증 환자 다시 500명대로…신규 확진 12만9411명',
   '20일 0시 기준…위중증 511명, 사망 84명누적 확진자 2212만9387명 집계\n\n\n\n19일 서울 마포구 보건소 코로나19 선별진료소를 찾은 시민들이 검사를 받기 위해 차례를 기다리고 있다. 연합뉴스코로나19 신규 확진자가 12만명대로 집계된 가운데, 400명대였던 위중증 환자가 500명대로 다시 늘었다.20일 중앙방역대책본부(방대본)는 0시 기준 신규 확진자가 12만9411명(국내 발생 12만8919명, 해외 유입 492명)이라고 밝혔다. 전날(국내 발생 13만8347명, 해외유입 465명)보다 국내 발생 환자는 줄었으나 해외 유입 환자는 늘었다. 국내 발생 환자 가운데 수도권에서 5만7400명(44.5%), 비수도권에서 7만1519명(55.5%)이 발생했다. 최근 1주간 국내에서 발생한 하루 평균 확진자는 약 12만7112명이다. 누적 확진자는 2212만9387명이다. 위중증 환자는 전날보다 19명 늘어난 511명이다. 최근 1주일간 위중증 환자는 14일부터 512명→521명→563명→469명→470명→492명으로, 500명대에서 400명대로 잠시 줄었다가 이날 다시 500명대로 늘었다. 입원한 위중증 환자 가운데 86.7%(443명)가 60살 이상이다. 위중증 병상은 전날 오후 5시를 기준으로 전국 1801개 중 805개를 사용 중으로, 가동률은 44.7%다. 사망자는 전날보다 1명 증가한 84명이다. 이 가운데 81명이 60살 이상(96.4%)이다. 최근 1주일간 사망자는 14일부터 57명→50명→37명으로 소폭 줄다가 17일부터 42명→61명→83명으로 늘기 시작했다. 누적 사망자는 2만5980명(치명률 0.12%)다.이날 0시 기준 재택치료자는 모두 74만4790명이다. 24시간 운영되는 재택치료 의료상담센터

In [17]:
dict_= keyword_dicts[-2].copy()
load_path = f'./{datetime_text}/img/{dict_["img_name"]}'
save_path = f'./{datetime_text}/segmentation/{load_path.split("/")[-1]}'
if dict_['namuKeyword_kind'] == 'person':
  try:
    temp_img = segsave.segSave(model_present, load_path, save_path=None)
    temp_img = np.stack([temp_img[:,:,2], temp_img[:,:,1], temp_img[:,:,0], temp_img[:,:,3]], axis=2)
    dict_['img'] = segsave.makeWhiteEdge(temp_img, save_path=save_path, iterations=5)
  except:
    pass
# elif dict_['namuKeyword_kind'] == '-':
  # continue   
else:
  try:
    temp_img = cv2.imread(load_path, cv2.IMREAD_UNCHANGED)
    
    try:
      temp_img = np.stack([temp_img[:,:,2], temp_img[:,:,1], temp_img[:,:,0], temp_img[:,:,3]], axis=2)
      dict_['img'] = segsave.makeWhiteEdge(temp_img, save_path=save_path, iterations=5)
    except:
      shutil.copy(load_path, save_path)
      dict_['img'] = temp_img
  except:
    pass

In [18]:
temp_img.shape

NameError: ignored

In [ ]:
dict_['img']

In [ ]:
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
hh = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

import requests
import re
import os 
import numpy as np
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image

header={"user-agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

# def scrapNamuImg(key, path, namuKeyword_kind='person'):
#     driver = scrap.set_chrome_driver()

#     url = 'https://namu.wiki/w/'+key
#     driver.get(url=url)
#     html = driver.page_source
#     driver.close()
#     soup = BeautifulSoup(html, 'lxml')

#     if namuKeyword_kind == 'person':
#       imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) if not d.find_parent('dd')]
#       for i, link in enumerate(imglink):
#           res=requests.get("https:"+link,headers=header)
#           # print(i)
#           if 'svg' not in res.text and 'video' not in res.text:
#               urlopen_img = Image.open(BytesIO(res.content))
#               counts = np.unique(np.array(urlopen_img.split()[-1]), return_counts=True)[1]
#               ratio = counts[0]/np.sum(counts)
#               if urlopen_img.size[1]*urlopen_img.size[0] > 100000 and ratio<0.3 :
#                   print(res.url)
#                   urlopen_img.save(path,'png')
#                   break
#     else :
#       imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) if not d.find_parent('dd') and ('로고' in  d['alt'] or'logo' in  d['alt'] or 'CI' in  d['alt'] or 'ci' in  d['alt'])]
#       for i, link in enumerate(imglink):
#           res=requests.get("https:"+link,headers=header)
#           # print(i)
#           if 'svg' in res.text:
#               print(res.url)
#               urlopen_img = Image.open(BytesIO(cairosvg.svg2png(res.content)))
#               urlopen_img.save(path,'png')
#               break
#           else:
#               print(res.url)
#               urlopen_img = Image.open(BytesIO(res.content))
#               urlopen_img.save(path,'png')
#               break

In [ ]:
namuKeyword_kind = 'person'
key = '카세미루'
path = 'asdasd'


driver = scrap.set_chrome_driver()

url = 'https://namu.wiki/w/'+key
driver.get(url=url)
html = driver.page_source
driver.close()
soup = BeautifulSoup(html, 'lxml')

if namuKeyword_kind == 'person':
  imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) if not d.find_parent('dd')]
  for i, link in enumerate(imglink):
      res=requests.get("https:"+link,headers=header)
      if 'svg' not in res.text and 'video' not in res.text:
          urlopen_img = Image.open(BytesIO(res.content))
          counts = np.unique(np.array(urlopen_img.split()[-1]), return_counts=True)[1]
          ratio = counts[0]/np.sum(counts)
          if urlopen_img.size[1]*urlopen_img.size[0] > 100000 and ratio<0.3 :
              print(res.url)
              urlopen_img.save(path,'png')
              break
else :
  imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) 
                    if not d.find_parent('dd') and (key in  d['alt']) and
                      ('로고' in  d['alt'] or'logo' in  d['alt']or 'CI' in  d['alt'] or 'ci' in  d['alt'] or '휘장' in d['alt']) ]
  if len(imglink) < 1:
    imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) 
                      if not d.find_parent('dd') and ('로고' in  d['alt'] or'logo' in  d['alt']or 'CI' in  d['alt'] or 'ci' in  d['alt'] or '휘장' in d['alt']) ]
  for i, link in enumerate(imglink):
      res=requests.get("https:"+link,headers=header)
      # print(i)
      try:
          print(res.url)
          urlopen_img = Image.open(BytesIO(cairosvg.svg2png(res.content)))
          urlopen_img.save(path,'png')
          break
      except:
          print(res.url)
          urlopen_img = Image.open(BytesIO(res.content))
          urlopen_img.save(path,'png')
          break


In [ ]:
html

In [ ]:
soup.find_all('img', attrs={'class':'dVTtICxy'})

In [ ]:
[d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'})]

In [ ]:
re = 'https://w.namu.la/s/c9c48bc9a0e156492e1c9d388ac58dbcec8710ac8cc3d10da9cbd0254018cfc26d66860c68e72a1c439604c4e28ea7f2fa00026ec52bd2327cd6f903a64f477ff2ce84521e0bde7d58528ecee15268a5a6906c8403edbf977d586e5df990438b'
res=requests.get(re,headers=header)

In [ ]:
res=requests.get(re,headers=header)

In [ ]:
img = Image.open(BytesIO(cairosvg.svg2png(res.content)))

In [ ]:
import cv2
img_arr = np.array(img)
im_alpha = img_arr[:,:,3]
kernel = np.ones((3,3), np.uint8)
result = cv2.dilate(im_alpha, kernel, iterations=3)

white_space_mask = np.stack([np.where(result - im_alpha>0, True,False)]*3, axis=2)

img_arr[:,:,3] = result
img_arr[:,:,:3][white_space_mask] = 255

In [ ]:
Image.fromarray(img_arr).save('/content/drive/MyDrive/Colab Notebooks/collage/result/9__.png','png')